In [ ]:
# insatll the following packages
# pip install spacy
# pip install PyPDF2
# pip install requests

In [42]:
import re
import pandas as pd
import spacy
from spacy.matcher import PhraseMatcher
import docx2txt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.probability import FreqDist
from nltk.stem import WordNetLemmatizer
from pprint import pprint
from bs4 import BeautifulSoup
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from io import StringIO
import bs4 as bs
from pathlib import Path
from collections import Counter

In [2]:
# Load English language model for NLP
nlp = spacy.load('en_core_web_lg')

In [3]:
# Define regular expressions for email and phone number extraction
email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
phone_pattern = r'\b(\+\d{1,2}\s?)?1?\-?\.?\s?(\()?[\d\s]{3}(\)?[\s.-]?)?[\d\s]{3}[\s.-]?[\d\s]{4}\b'

In [ ]:
# Initialize Matcher for matching skill keywords
#matcher = PhraseMatcher(nlp.vocab)
#skill_keywords = ['Python', 'Machine Learning', 'Deep Learning', 'TensorFlow', 'Keras', 'Pandas',
                  #'NumPy', 'scikit-learn', 'Computer Science', 'Data Visualization', 'AWS', 'Git']
#skill_patterns = [nlp(keyword.lower()) for keyword in skill_keywords]
#matcher.add('Skills', None, *skill_patterns)

In [4]:
# Parse resume text using NLP
def parse_resume(text):
    doc = nlp(text)

    # Extract email
    email = re.search(email_pattern, text)
    email = email.group() if email else None

    # Extract phone number
    phone = re.search(phone_pattern, text)
    phone = phone.group() if phone else None

    # Extract skills
    #skills = [doc[start:end].text for match_id, start, end in matcher(doc)]
    #skills = [doc[start:end].text for skill in skills]

    # Sections to extract
    sections = ['Profile','Projects','Skills', 'Education', 'Professional Experience', 'Languages']
    
    info = {}
    for section in sections:
        result = re.search(rf'{section}(.*?)(?:\n[A-Z]|$)', text, re.DOTALL)
        info[section] = result.group(1).strip() if result else None

    return email, phone, info['Skills'], info['Profile'], info['Projects'], info['Education'], info['Professional Experience'], info['Languages']

In [5]:

# Read the resume file (assuming it's in the same directory)
resume_file = './data/Shivani_CV_2.docx'
resume_file

'./data/Shivani_CV_2.docx'

In [6]:
# Extract text from the docx file
resume_text = docx2txt.process(resume_file)
resume_text

'REKALA SHIVANI\n\n\n\n\tBerlin,Germany\t rekala.shivani@gmail.com    49 15145983960      LinkedIn    Github\n\n\n\n\n\nProfile\n\n\n\n\n\nI am a data scientist experience in building data-intensive applications. In addition to my academic background in mathematics and computer science, I possess extensive experience with Python programming and machine learning tools. I enjoy learning new programming languages and keeping up with technological advances.\n\n\n\nProjects\n\n\n\n\n\nOnline Movie Recommender, Python, Pandas, Numpy, Scikit-learn, Collaborative filtering models, Streamlit, Render\n\n\n\nBuilt online movie recommender system using collaborative filtering algorithms and user-based similarity metrics to provide personalized movie recommendations. Launched a web app with Streamlit and deployed on Render.\n\n\n\nTime Series Analysis, Python, Pandas, Numpy, Matplotlib, Seaborn, Scikit-learn, statsmodels\n\n\n\nPerformed time series analysis on temperature data, applied statsmodels

In [7]:
# Remove line breaks and multiple spaces
resume_text = re.sub(r'\n+', ' ', resume_text)
resume_text = re.sub(r' +', ' ', resume_text)

In [8]:
# Call the resume parser function
email, phone, skills, profile, projects, education, experience, languages = parse_resume(resume_text)

cv_doc = {
    "Email": email,
    "Phone": phone,
    "Skills": skills,
    "Profile": profile,
    "Projects": projects,
    "Education": education,
    "Professional Experience": experience,
    "Languages": languages
}


In [9]:
cv_doc

{'Email': 'rekala.shivani@gmail.com',
 'Phone': '15145983960',
 'Skills': 'Data Visualisation, Data analysis, Matplotlib, Seaborn, Plotly, Web Scraping, Linear Algebra, Analysis, Python, PostgreSQL, SQL, Pandas, Numpy Bash, Git, Cloud computing, AWS, EC2, RDS, Docker Machine Learning, TensorFlow, Keras, Scikit-learn, Deep Learning, Natural language Processing(NLP) Education Spiced Academy, Data Science Bootcamp 03/2023 – 06/2023 Intensive on-site, three-months, python-based Data Science Bootcamp with weekly projects Berlin, Germany led by domain experts while using the most in-demand technologies. Osmania University, Masters of Business Administration 06/2012 – 08/2014 Human Resources Management Hyderabad, India Jawaharlal Nehru Technology University , Bachelor of Technology 08/2007 – 06/2011 Electronica and Communication Engineering. Hyderabad, India Professional Experience Talent Manager, MoBerries 05/2019 – 11/2022 Identified and sourced top candidates through resume analysis on the

In [10]:
cv_string = str(cv_doc)
print(cv_string)

{'Email': 'rekala.shivani@gmail.com', 'Phone': '15145983960', 'Skills': 'Data Visualisation, Data analysis, Matplotlib, Seaborn, Plotly, Web Scraping, Linear Algebra, Analysis, Python, PostgreSQL, SQL, Pandas, Numpy Bash, Git, Cloud computing, AWS, EC2, RDS, Docker Machine Learning, TensorFlow, Keras, Scikit-learn, Deep Learning, Natural language Processing(NLP) Education Spiced Academy, Data Science Bootcamp 03/2023 – 06/2023 Intensive on-site, three-months, python-based Data Science Bootcamp with weekly projects Berlin, Germany led by domain experts while using the most in-demand technologies. Osmania University, Masters of Business Administration 06/2012 – 08/2014 Human Resources Management Hyderabad, India Jawaharlal Nehru Technology University , Bachelor of Technology 08/2007 – 06/2011 Electronica and Communication Engineering. Hyderabad, India Professional Experience Talent Manager, MoBerries 05/2019 – 11/2022 Identified and sourced top candidates through resume analysis on the A

In [11]:
#To extract keywords from a resume file
def extract_keywords_from_doc(doc_file):
    doc_text = ""
    doc = Document(doc_file)
    for paragraph in doc.paragraphs:
        doc_text += paragraph.text

    return extract_keywords(doc_text)

def extract_keywords(text):

    # Tokenize the resume text into words
    words = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word.lower() for word in words if word.lower() not in stop_words]

    # Lemmatize words to their base form
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]

    # Get keywords by filtering out non-alphabetic words
    cv_keywords = [word for word in words if word.isalpha()]

    return cv_keywords

In [13]:
cv_keywords = extract_keywords(cv_string)
#print("CV Keywords:", cv_keywords)
cv_keywords

['visualisation',
 'data',
 'analysis',
 'matplotlib',
 'seaborn',
 'plotly',
 'web',
 'scraping',
 'linear',
 'algebra',
 'analysis',
 'python',
 'postgresql',
 'sql',
 'panda',
 'numpy',
 'bash',
 'git',
 'cloud',
 'computing',
 'aws',
 'rds',
 'docker',
 'machine',
 'learning',
 'tensorflow',
 'kera',
 'deep',
 'learning',
 'natural',
 'language',
 'processing',
 'nlp',
 'education',
 'spiced',
 'academy',
 'data',
 'science',
 'bootcamp',
 'intensive',
 'data',
 'science',
 'bootcamp',
 'weekly',
 'project',
 'berlin',
 'germany',
 'led',
 'domain',
 'expert',
 'using',
 'technology',
 'osmania',
 'university',
 'master',
 'business',
 'administration',
 'human',
 'resource',
 'management',
 'hyderabad',
 'india',
 'jawaharlal',
 'nehru',
 'technology',
 'university',
 'bachelor',
 'technology',
 'electronica',
 'communication',
 'engineering',
 'hyderabad',
 'india',
 'professional',
 'experience',
 'talent',
 'manager',
 'moberries',
 'identified',
 'sourced',
 'top',
 'candidate

## extract keywords from job description

In [30]:
# load job description html file
with open("./data/Data Scientist _AI.html", "r") as file:
    job_post= file.read()
job_post

'<!DOCTYPE html><html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://opengraph.org/schema/" style="height: 100%;" lang="en" ><head><link href="/jobView/static/css/gd-jobView.4dffe.css" rel="stylesheet" media="all"/><meta  charset="utf-8"/><meta  name="referrer" content="origin"/><meta  name="theme-color" content="#0caa41"/><meta  name="msapplication-navbutton-color" content="#0caa41"/><meta  name="apple-mobile-web-app-capable" content="yes"/><meta  name="apple-mobile-web-app-status-bar-style" content="#0caa41"/><meta  http-equiv="Cache-Control" content="no-cache,must-revalidate"/><meta  http-equiv="Pragma" content="no-cache"/><meta  http-equiv="Expires" content="0"/><meta  name="viewport" content="width=device-width, initial-scale=1.0"/><link  rel="manifest" href="/manifest.json"/><link  rel="shortcut icon" href="/favicon.ico" type="image/vnd.microsoft.icon"/><link  rel="icon" href="/favicon.ico" type="image/x-icon"/><link  rel="preconnect" href="https://fonts.gstatic.co

In [15]:
#parse the job description html file

#def parse_html(job_post):
    
#    soup = bs.BeautifulSoup(job_post, "html.parser")

#    title = soup.find('div', {'data-test':'job-title'}).get_text()

#    location = soup.find("span", {'data-test':'location'}).get_text()

#    company_name = soup.find("div", class_="d-flex align-items-center justify-content-between css-16nw49e e11nt52q1").get_text()

#    job_description = soup.find("div", id="JobDescriptionContainer").get_text()
    
#    jobs= {
        "title": title,
        "location": location,
        "company_name": company_name,
        "job_description": job_description
#    }

#    return jobs

In [113]:
def extract_keywords_from_url(url):

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(job_post, 'html.parser')

    #title = soup.find('div', {'data-test':'job-title'}).get_text()

    #location = soup.find("span", {'data-test':'location'}).get_text()

    #company_name = soup.find("div", class_="d-flex align-items-center justify-content-between css-16nw49e e11nt52q1").get_text()

    # Find the element containing the job description
    job_description_element = soup.find('div', {'id': 'JobDescriptionContainer'})

    # Extract the text from the job description element
    if job_description_element is not None:
        job_description = job_description_element.get_text()
    
    # List of stop words in English
    stop_words = set(stopwords.words('english'))

    # Tokenize the text and remove stop words
    words = job_description.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]

    # Rejoin the words into a string without stop words
    filtered_description = ' '.join(filtered_words)
    
    return filtered_description

In [114]:
job_description = extract_keywords_from_url(filtered_description)
pprint(filtered_description)

('customers expect technology solutions? next level! team FERCHAU succeeds day '
 'day. looking you: someone ambitious who, like us, wants take technology next '
 'level. involved exciting projects well-known car industry OEMs component '
 'suppliers advance digitalisation networking. one pull wool eyes many years '
 'experience. oversee digital transformation start know exactly impact '
 'economic development future. would like stay up-to-date always interested '
 'new developments. take challenges application artificial intelligence, '
 'machine learning, data mining, predictive analytics, ETL jobs, optimization '
 'processes data pipelining / streaming, usage analytical databases '
 'development analytical services cloud environment preferably AWS, especially '
 'area analytical tooling (e.g. S3, EC2, Data Pipeline, CloudWatch, '
 'CloudFormation) gitlab working business stakeholders (incl. gathering '
 'translating business requirements technical solutions, explaining technical '
 

In [ ]:
from rake_nltk import Rake

r = Rake()
r.extract_keywords_from_text(filtered_description)
keywords = r.get_ranked_phrases()

# To extract keyword phrases from a job description
#for rating, keyword in r.get_ranked_phrases_with_scores():
#    if rating > 5:
#        print(rating, keyword)

# To get keywords with by splitting the phrases
#print(set([keyword for keyword in keywords if len(keyword.split()) > 1]))
#jd_keywords = [keyword for keyword in keywords if len(keyword.split()) < 3]

# Extract individual keywords
jd_keywords = set(keyword for keyword in keywords if len(keyword.split()) < 3)

jd_keywords

In [96]:
# Split the keywords further
split_keywords = []
for keyword in single_keywords:
    split_keywords.extend(keyword.split())

split_keywords

['like',
 'us',
 'etl',
 'jobs',
 'etl',
 'jobs',
 'data',
 'pipeline',
 'data',
 'pipeline',
 'data',
 'mining',
 'data',
 'mining',
 'next',
 'level',
 'technical',
 'audience',
 'technical',
 'audience',
 'tobias',
 'fleßner',
 'someone',
 'ambitious',
 'show',
 'report',
 'predictive',
 'analytics',
 'predictive',
 'analytics',
 'machine',
 'learning',
 'machine',
 'learning',
 'ideation',
 'deployment',
 'deliver',
 'production',
 'bmo',
 'mr',
 'end',
 'streaming',
 'streaming',
 's3',
 's3',
 'microsoft',
 'looking',
 'infrastructure',
 'infrastructure',
 'incl',
 'incl',
 'g',
 'g',
 'g',
 'ec2',
 'ec2',
 'e',
 'e',
 'e',
 'code',
 'code',
 'cloudwatch',
 'cloudwatch',
 'cloudformation',
 'cloudformation',
 'ci',
 'ci',
 'apply',
 '14535']

In [100]:
# Function to extract keywords from a text (e.g. job description)
#def extract_keywords(text):
    # Lowercasing the text
#    text = text.lower()
    
    # Removing special characters and digits
#    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Tokenizing the text
#    words = word_tokenize(text)
    
    # Removing stopwords
#    stop_words = set(stopwords.words("english"))
#    filtered_words = [word for word in words if word not in stop_words]
    
    # Counting word frequencies
#    word_frequencies = Counter(filtered_words)
    
    # Sorting the words based on frequency
#    sorted_words = sorted(word_frequencies.items(), key=lambda x: x[1], reverse=True)
    
    # Extracting top N keywords
    #N = 10  # You can change this value
#    keywords = [word for word, freq in sorted_words]
    
#    return keywords

In [101]:
# Extracting keywords
#job_keywords = extract_keywords(job_description)
#job_keywords

,title,location,company_name,job_description
1,Data Scientist - AI (m/f/d),Berlin,FERCHAU3.7★,What do our customers expect from their techno...


In [104]:
#job_keywords = extract_keywords(job_post)

In [ ]:
#job_keywords = extract_keywords_from_url(job_post)
#job_keywords

In [102]:
# Count keyword frequencies in the job description
#job_keyword_counts = Counter(job_keywords)
#job_keyword_counts

# Convert the job keywords into a dataframe
#job_keywords_df = pd.DataFrame.from_dict(job_keyword_counts, orient='index', columns=['count'])
#job_keywords_df.sort_values(by=['count'], ascending=True, inplace=True)
#job_keywords_df

In [59]:
# Function to calculate keyword weight based on frequency in job description
#def calculate_weight(keyword, keyword_counts):
    # Weight is simply the frequency of the keyword in the job description
#    return keyword_counts.get(keyword, 0)

In [60]:
# Calculate weights for keywords in the job description
#job_keyword_weights = {}

#for keyword in job_keywords:
#    job_keyword_weights[keyword] = calculate_weight(keyword, job_keyword_weights)

In [ ]:
# Find the keywords that are missing in the resume
#missing_keywords = {k: v for k, v in job_keyword_weights.items() if k not in cv_keywords}
#missing_keywords

In [ ]:
# Suggest adding the missing keywords to the resume
#print("Consider adding the following keywords to your resume:")
#for keyword, weight in missing_keywords.items():
#    print(f"- {keyword} (Weight: {weight})")

In [ ]:
# Example of how you could modify the resume text (you should actually do this thoughtfully and manually)
#updated_resume_text = resume_text + " " + " ".join(missing_keywords.keys())

# Output updated resume (for demonstration purposes only)
#print("\nUpdated Resume (for demonstration purposes only):")
#updated_resume_text
#save the updated resume
#with open("./data/updated_resume.doc", "w") as file:
#    file.write(updated_resume_text)

# similarity score TFIDF

In [106]:
# Combine them into a list of documents
documents = [cv_string, filtered_description]
#documents = [cv_keywords, job_keywords]
#documents

In [107]:
# Initialize the TF-IDF vectorizer
vectorizer = TfidfVectorizer(stop_words="english", max_features=1000, max_df=0.5, smooth_idf=True)

# Compute the TF-IDF scores
tfidf_matrix = vectorizer.fit_transform(documents)

# Convert the matrix to an array
tfidf_array = tfidf_matrix.toarray()

# Extract the feature names (the words)
feature_names = vectorizer.get_feature_names_out()

# The TF-IDF scores of the resume
resume_scores = dict(zip(feature_names, tfidf_array[0]))

# The TF-IDF scores of the job description
job_description_scores = dict(zip(feature_names, tfidf_array[1]))

# Create a DataFrame
data = {
    'Word': feature_names,
    'Resume_TFIDF': tfidf_array[0],
    'JobDescription_TFIDF': tfidf_array[1]
}

In [108]:
df = pd.DataFrame(data)
df

,Word,Resume_TFIDF,JobDescription_TFIDF
0,03,0.046271,0.000000
1,05,0.173518,0.000000
2,06,0.254493,0.000000
3,08,0.092543,0.000000
4,11,0.115678,0.000000
...,...,...,...
379,weekly,0.046271,0.000000
380,willing,0.000000,0.045268
381,wool,0.000000,0.045268
382,working,0.000000,0.226339


In [109]:
# Sort the DataFrame based on the JobDescription_TFIDF column in descending order
sorted_df = df.sort_values(by='JobDescription_TFIDF', ascending=False)
sorted_df

,Word,Resume_TFIDF,JobDescription_TFIDF
36,analytical,0.000000,0.271607
348,technical,0.000000,0.271607
382,working,0.000000,0.226339
209,like,0.000000,0.226339
347,team,0.000000,0.181071
...,...,...,...
236,needa,0.057839,0.000000
237,nehru,0.046271,0.000000
76,classifier,0.046271,0.000000
240,networks,0.046271,0.000000


In [110]:
# save the resume and job description dataframes to csv files
#sorted_df.to_csv('./data/similarity_score.csv', index=False)

In [111]:
# Calculate the matching score for each keyword
df['Matching_Score'] = df[['Resume_TFIDF', 'JobDescription_TFIDF']].min(axis=1)

# Sort the DataFrame based on the Matching_Score column in descending order
sorted_df = df.sort_values(by='JobDescription_TFIDF', ascending=False)
sorted_df


# Printing the sorted DataFrame (you might want to print only the top few rows)
#print(sorted_df.head(10))

,Word,Resume_TFIDF,JobDescription_TFIDF,Matching_Score
36,analytical,0.000000,0.271607,0.0
348,technical,0.000000,0.271607,0.0
382,working,0.000000,0.226339,0.0
209,like,0.000000,0.226339,0.0
347,team,0.000000,0.181071,0.0
...,...,...,...,...
236,needa,0.057839,0.000000,0.0
237,nehru,0.046271,0.000000,0.0
76,classifier,0.046271,0.000000,0.0
240,networks,0.046271,0.000000,0.0


## modify resume based on skillsets, projects and job description

In [50]:
#load the skills and projects files
ds_projects = pd.read_excel("./data/projects.xlsx")
ds_projects

,Title,Skills,Description
0,The Gapminder Project,"Bash, Git, Pandas, descriptive statistics, mat...","Contributed to the Gapminder project, analyzin..."
1,Titanic classification,"python, numpy, scikit-learn, matplotlib, seabo...","Analyzed Titanic dataset, identifying factors ..."
2,Bike Sharing project,"python, pandas, numpy, scikit-learn, matplotli...",Utilized bike sharing data to analyze usage pa...
3,Text Classification,"web scrapping, natural language processing, N...","Performed text classification on lyrics data, ..."
4,Metabase Dashboard,"SQL, PostgresSQL, cloud computing, AWS, EC2, RDS","Built Northwind metadata dashboard, showcasing..."
5,The Data pipeline,"APIs, Docker, MongoDB, ETL","Built Reddit data pipeline, enabling automated..."
6,Time series Analysis,"statsmodels, markov chains",Performed time series analysis on temperature ...
7,Supermarket simulation,"program design, python",Employed statistical analysis techniques to an...
8,Real-time image classifer,"deep learning, artifical neural network(ANN),c...",Developed a real-time image classifier using d...
9,Online Movie Recommender,"collaborative filtering, Render, Streamlit, co...",Built online movie recommender system using co...


In [51]:
ds_skills = pd.read_excel("./data/ds_skills.xlsx")
ds_skills

,skills
0,data analysis
1,Matplotlib
2,Plotly
3,Seaborn
4,Data Storytelling
...,...
101,Stakeholder management
102,Empirical mindset
103,Storytelling with data
104,Cross-functional collaboration


In [52]:
def suggest_projects(job_description, ds_projects):
    # Create a TfidfVectorizer to convert text into numerical features
    vectorizer = TfidfVectorizer(stop_words="english", max_features=1000, max_df=0.5, smooth_idf=True)

    # Fit and transform the project data to numerical features
    project_features = vectorizer.fit_transform(ds_projects['Skills'] + ' ' + ds_projects['Description'])
    #print(vectorizer.get_feature_names_out())
    
    # Transform the job description into numerical features
    job_features = vectorizer.transform([job_description])
    #print(pd.DataFrame(job_features.todense(), columns=vectorizer.get_feature_names_out()))
    
    # Compute the cosine similarity between the job description and project data
    similarity_scores = cosine_similarity(job_features, project_features).flatten()

    # Sort the projects based on similarity scores
    sorted_indices = similarity_scores.argsort()[::-1]

    # Return the top 4 relevant projects
    top_projects = ds_projects.iloc[sorted_indices][:4]

    # Return the top 4 relevant project titles skills and discritpiton and convert into a dataframe
    return top_projects[['Title', 'Skills', 'Description']]


In [53]:
job_description_ = job_description
suggested_projects = suggest_projects(job_description_, ds_projects)
print("Suggested projects:")
pprint(suggested_projects)

Suggested projects:
                       Title  \
5          The Data pipeline   
2       Bike Sharing project   
8  Real-time image classifer   
4         Metabase Dashboard   

                                              Skills  \
5                         APIs, Docker, MongoDB, ETL   
2  python, pandas, numpy, scikit-learn, matplotli...   
8  deep learning, artifical neural network(ANN),c...   
4   SQL, PostgresSQL, cloud computing, AWS, EC2, RDS   

                                         Description  
5  Built Reddit data pipeline, enabling automated...  
2  Utilized bike sharing data to analyze usage pa...  
8  Developed a real-time image classifier using d...  
4  Built Northwind metadata dashboard, showcasing...  
